In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from functools import partial
from optim_correspondance import mp_optimize_homography
from multiprocessing import Pool, cpu_count

from utils.tools import load_files_from_dir

path_to_files = Path('rawData') / 'calib'

left = load_files_from_dir(path_to_files / 'pan')
right = load_files_from_dir(path_to_files / 'mono')

print(f"List of keys in dict: {list(right.keys())}\n")
print(f"Number of left frames: {len(left['time_ns']):,}")
print(f"Number of right frames: {len(right['time_ns']):,}")

Fix timing between stereo cameras

In [ ]:
MAX_TIME_DIFF = 0.001  # seconds

def _get_diff(left, right):
    if len(left['time_ns']) > len(right['time_ns']):
        return np.abs(left['time_ns'][:, None] - right['time_ns'][None, :])
    else:
        return np.abs(left['time_ns'][None, :] - right['time_ns'][:, None])
diff_before = _get_diff(left, right).argmin(0)

# Remove frames that are not in both cameras
while abs(len(left['time_ns']) - len(right['time_ns'])) > 1:
    # Find the difference between the timestamps of the two cameras
    diff = _get_diff(left, right)

    # Find the indices in the longer array that have no counterpart in the shorter array
    diff_values_of_longer_dict = np.min(diff, axis=1)
    mask_longer_dict = diff_values_of_longer_dict <= (MAX_TIME_DIFF * 1e9)
    indices_of_longer_dict = np.where(mask_longer_dict)[0]
    if len(left['time_ns']) > len(right['time_ns']):
        left = {k: v[indices_of_longer_dict] for k, v in left.items()}
    else:
        right = {k: v[indices_of_longer_dict] for k, v in right.items()}
print(f"Length of left: {len(left['time_ns']):,}")
print(f"Length of right: {len(right['time_ns']):,}")

# Sort both arrays to have the closest timestamps have same index
diff = np.argmin(np.abs(left['time_ns'][:, None] - right['time_ns'][None, :]), axis=0)
left = {k: v[diff] for k, v in left.items()}

diff = np.abs(left['time_ns'] - right['time_ns'])*1e-9
print(f"Max time difference between frames: {np.max(diff):.2g} seconds")
print(f"Min time difference between frames: {np.min(diff):.2g} seconds")

plt.figure()
plt.scatter(range(len(left['time_ns'])), left['time_ns'] * 1e-9, label='Left', s=2)
plt.scatter(range(len(right['time_ns'])), right['time_ns'] * 1e-9, label='Right', s=2)
plt.title('Timestamps of frames')
plt.legend()
plt.grid()
plt.show()

plt.figure()
plt.scatter(range(len(diff)), diff, c='r', s=2)
plt.plot(np.ones_like(left['time_ns'])*np.mean(diff))
plt.grid()
plt.title('Time difference between frames')
plt.ylabel('Time difference [s]')
plt.show()



# Save each frame in a separate file

In [ ]:
# Delete old files
for p in path_to_files.glob('*.npy'):
    p.unlink()

for idx, frame in enumerate(left['frames']):
    np.save(path_to_files / f'src_{idx}.npy', frame)
for idx, frame in enumerate(right['frames']):
    np.save(path_to_files / f'dest_{idx}.npy', frame)
print(f"Saved {len(left['frames'])} frames to {path_to_files}")

# Optimize the homography

In [ ]:
DISTANCE_FROM_FRAME_EDGES = 10
LOSS_THRESHOLD_GRAY_LEVELS = 50

list_of_files = list(path_to_files.glob('src_*.npy'))
list_of_files.sort(key=lambda x: int(x.stem.split('src_')[1]))

# Remove all points files
for path in tqdm(path_to_files.glob('points_*.csv'), desc='Remove all points files'):
    path.unlink()
for path in tqdm(path_to_files.glob('M_*.npy'), desc='Remove all homography files'):
    path.unlink()

optimizer = partial(mp_optimize_homography, distance_from_frame_edges=DISTANCE_FROM_FRAME_EDGES,
                    loss_threshold=LOSS_THRESHOLD_GRAY_LEVELS, path_to_files=path_to_files)

# List of files -> list of indices
list_of_indices = list(map(lambda x: int(x.stem.split('src_')[1]), list_of_files))

# Optimize points
with Pool(cpu_count()) as pool:
    ret_vals = list(tqdm(pool.imap(optimizer, list_of_indices), total=len(list_of_indices), desc='Optimizing points'))
ret_vals = filter(lambda x: x is not None, ret_vals)
ret_vals = list(filter(lambda x: x <= LOSS_THRESHOLD_GRAY_LEVELS, ret_vals))
print(f'Number of frames with loss bellow threshold: {len(ret_vals)}')